# Import Libraries

In [ ]:
print("Importing required libraries...")
import pandas as pd
print("Succesfully imported pandas library")
import datetime
print("Succesfully imported datetime library")
import time
print("Succesfully imported time library")
import pytz
print("Succesfully imported pytz library")
print("Installing WazeRouteCalculator library")
!pip install WazeRouteCalculator
print("Succesfully installed WazeRouteCalculator library")
import WazeRouteCalculator
print("Succesfully imported WazeRouteCalculator library")
print("All imports completed!")

# Define Values

In [ ]:
print("Importing required libraries...")
uberdata_url = 'https://docs.google.com/spreadsheets/d/19DnHil6Iar2YV67hUU36VXbvKMYEQORzWHrE7LZOGpA/edit#gid=712533051'
print(f"Google Sheets URL loaded: {uberdata_url}")
uberdata = uberdata_url.replace('/edit#gid=', '/export?format=csv&gid=')
print(f"Google Sheets URL changed to: {uberdata}")
datetime_format = '%Y-%m-%d|%H:%M:%S'
print(f"Date and time format changed to {datetime_format}")
timezone = pytz.timezone('Asia/Bangkok')
print(f"Time Zone set to {timezone}")
start_row = 190000 # Index number of dataframe to process
print(f"Start row set to {start_row}")
rows_per_chunk = 1000 # Number of rows to process at a time
print(f"Chunk size set to {rows_per_chunk}")
timeout = 10  # Wait duration for request timeout in seconds
print(f"Timeout duration set to {timeout}")
max_retry = 10 # Number of retries before skipping a row
print(f"Maximum retries set to {max_retry}")
print("All values defined!")


# Run the codes

## 1st Iteration

In [ ]:
end_row = start_row+rows_per_chunk # Value of end_row to enable resuming from last cell

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Loading CSV into DataFrame...")
df_uberdata = pd.read_csv(uberdata)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - CSV is loaded")
df_uberdata.insert(0, 'no', df_uberdata.reset_index().index + 1)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Added row index column")
df_uberdata = df_uberdata.iloc[start_row:end_row].reset_index(drop=True)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Starting processing rows {start_row} - {end_row}")

# Define a function to calculate the driving distance between two coordinates
def calculate_trip_distance(row):
    current_time = datetime.datetime.now(timezone).strftime(datetime_format)
    print(f"{current_time} - Calculating distance for row {start_row+row.name+1}")
    pickup_coordinates = f"{row['pickup_latitude']}, {row['pickup_longitude']}"
    dropoff_coordinates = f"{row['dropoff_latitude']}, {row['dropoff_longitude']}"
    region = 'US'
    timeout_count = 0
    while True:
        try:
            route = WazeRouteCalculator.WazeRouteCalculator(pickup_coordinates, dropoff_coordinates, region)
            distance_km, travel_time = route.calc_route_info()  # Unpack the tuple
            return distance_km
        except Exception as e:
            if timeout_count == max_retry:
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred for {max_retry} times, skipping row {start_row+row.name+1}. {e}")
                return None
            else:
                timeout_count += 1
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred, retrying after {timeout} seconds, skipping row {start_row+row.name+1} after {max_retry-timeout_count+1} more retries. {e}")
                time.sleep(timeout)

# Apply the function to each row to calculate the trip distance and save it in a new column
df_uberdata['trip_distance'] = df_uberdata.apply(calculate_trip_distance, axis=1)

# Export the dataframe to a CSV file with the current time in the file name
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
output_file = f"uber_dataset_driving_distance_rows_{start_row}-{end_row}_{current_time}.csv"
df_uberdata.to_csv(output_file, index=False)

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Dataframe exported to {output_file}")
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Finished processing rows {start_row}-{end_row}")
    
# Move on to the next chunk
start_row = end_row

## 2nd Iteration

In [ ]:
end_row = start_row+rows_per_chunk # Value of end_row to enable resuming from last cell

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Loading CSV into DataFrame...")
df_uberdata = pd.read_csv(uberdata)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - CSV is loaded")
df_uberdata.insert(0, 'no', df_uberdata.reset_index().index + 1)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Added row index column")
df_uberdata = df_uberdata.iloc[start_row:end_row].reset_index(drop=True)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Starting processing rows {start_row} - {end_row}")

# Define a function to calculate the driving distance between two coordinates
def calculate_trip_distance(row):
    current_time = datetime.datetime.now(timezone).strftime(datetime_format)
    print(f"{current_time} - Calculating distance for row {start_row+row.name+1}")
    pickup_coordinates = f"{row['pickup_latitude']}, {row['pickup_longitude']}"
    dropoff_coordinates = f"{row['dropoff_latitude']}, {row['dropoff_longitude']}"
    region = 'US'
    timeout_count = 0
    while True:
        try:
            route = WazeRouteCalculator.WazeRouteCalculator(pickup_coordinates, dropoff_coordinates, region)
            distance_km, travel_time = route.calc_route_info()  # Unpack the tuple
            return distance_km
        except Exception as e:
            if timeout_count == max_retry:
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred for {max_retry} times, skipping row {start_row+row.name+1}. {e}")
                return None
            else:
                timeout_count += 1
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred, retrying after {timeout} seconds, skipping row {start_row+row.name+1} after {max_retry-timeout_count+1} more retries. {e}")
                time.sleep(timeout)

# Apply the function to each row to calculate the trip distance and save it in a new column
df_uberdata['trip_distance'] = df_uberdata.apply(calculate_trip_distance, axis=1)

# Export the dataframe to a CSV file with the current time in the file name
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
output_file = f"uber_dataset_driving_distance_rows_{start_row}-{end_row}_{current_time}.csv"
df_uberdata.to_csv(output_file, index=False)

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Dataframe exported to {output_file}")
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Finished processing rows {start_row}-{end_row}")
    
# Move on to the next chunk
start_row = end_row

## 3rd Iteration

In [ ]:
end_row = start_row+rows_per_chunk # Value of end_row to enable resuming from last cell

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Loading CSV into DataFrame...")
df_uberdata = pd.read_csv(uberdata)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - CSV is loaded")
df_uberdata.insert(0, 'no', df_uberdata.reset_index().index + 1)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Added row index column")
df_uberdata = df_uberdata.iloc[start_row:end_row].reset_index(drop=True)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Starting processing rows {start_row} - {end_row}")

# Define a function to calculate the driving distance between two coordinates
def calculate_trip_distance(row):
    current_time = datetime.datetime.now(timezone).strftime(datetime_format)
    print(f"{current_time} - Calculating distance for row {start_row+row.name+1}")
    pickup_coordinates = f"{row['pickup_latitude']}, {row['pickup_longitude']}"
    dropoff_coordinates = f"{row['dropoff_latitude']}, {row['dropoff_longitude']}"
    region = 'US'
    timeout_count = 0
    while True:
        try:
            route = WazeRouteCalculator.WazeRouteCalculator(pickup_coordinates, dropoff_coordinates, region)
            distance_km, travel_time = route.calc_route_info()  # Unpack the tuple
            return distance_km
        except Exception as e:
            if timeout_count == max_retry:
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred for {max_retry} times, skipping row {start_row+row.name+1}. {e}")
                return None
            else:
                timeout_count += 1
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred, retrying after {timeout} seconds, skipping row {start_row+row.name+1} after {max_retry-timeout_count+1} more retries. {e}")
                time.sleep(timeout)

# Apply the function to each row to calculate the trip distance and save it in a new column
df_uberdata['trip_distance'] = df_uberdata.apply(calculate_trip_distance, axis=1)

# Export the dataframe to a CSV file with the current time in the file name
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
output_file = f"uber_dataset_driving_distance_rows_{start_row}-{end_row}_{current_time}.csv"
df_uberdata.to_csv(output_file, index=False)

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Dataframe exported to {output_file}")
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Finished processing rows {start_row}-{end_row}")
    
# Move on to the next chunk
start_row = end_row

## 4th Iterartion

In [ ]:
end_row = start_row+rows_per_chunk # Value of end_row to enable resuming from last cell

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Loading CSV into DataFrame...")
df_uberdata = pd.read_csv(uberdata)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - CSV is loaded")
df_uberdata.insert(0, 'no', df_uberdata.reset_index().index + 1)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Added row index column")
df_uberdata = df_uberdata.iloc[start_row:end_row].reset_index(drop=True)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Starting processing rows {start_row} - {end_row}")

# Define a function to calculate the driving distance between two coordinates
def calculate_trip_distance(row):
    current_time = datetime.datetime.now(timezone).strftime(datetime_format)
    print(f"{current_time} - Calculating distance for row {start_row+row.name+1}")
    pickup_coordinates = f"{row['pickup_latitude']}, {row['pickup_longitude']}"
    dropoff_coordinates = f"{row['dropoff_latitude']}, {row['dropoff_longitude']}"
    region = 'US'
    timeout_count = 0
    while True:
        try:
            route = WazeRouteCalculator.WazeRouteCalculator(pickup_coordinates, dropoff_coordinates, region)
            distance_km, travel_time = route.calc_route_info()  # Unpack the tuple
            return distance_km
        except Exception as e:
            if timeout_count == max_retry:
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred for {max_retry} times, skipping row {start_row+row.name+1}. {e}")
                return None
            else:
                timeout_count += 1
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred, retrying after {timeout} seconds, skipping row {start_row+row.name+1} after {max_retry-timeout_count+1} more retries. {e}")
                time.sleep(timeout)

# Apply the function to each row to calculate the trip distance and save it in a new column
df_uberdata['trip_distance'] = df_uberdata.apply(calculate_trip_distance, axis=1)

# Export the dataframe to a CSV file with the current time in the file name
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
output_file = f"uber_dataset_driving_distance_rows_{start_row}-{end_row}_{current_time}.csv"
df_uberdata.to_csv(output_file, index=False)

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Dataframe exported to {output_file}")
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Finished processing rows {start_row}-{end_row}")
    
# Move on to the next chunk
start_row = end_row

## 5th Iteration

In [ ]:
end_row = start_row+rows_per_chunk # Value of end_row to enable resuming from last cell

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Loading CSV into DataFrame...")
df_uberdata = pd.read_csv(uberdata)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - CSV is loaded")
df_uberdata.insert(0, 'no', df_uberdata.reset_index().index + 1)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Added row index column")
df_uberdata = df_uberdata.iloc[start_row:end_row].reset_index(drop=True)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Starting processing rows {start_row} - {end_row}")

# Define a function to calculate the driving distance between two coordinates
def calculate_trip_distance(row):
    current_time = datetime.datetime.now(timezone).strftime(datetime_format)
    print(f"{current_time} - Calculating distance for row {start_row+row.name+1}")
    pickup_coordinates = f"{row['pickup_latitude']}, {row['pickup_longitude']}"
    dropoff_coordinates = f"{row['dropoff_latitude']}, {row['dropoff_longitude']}"
    region = 'US'
    timeout_count = 0
    while True:
        try:
            route = WazeRouteCalculator.WazeRouteCalculator(pickup_coordinates, dropoff_coordinates, region)
            distance_km, travel_time = route.calc_route_info()  # Unpack the tuple
            return distance_km
        except Exception as e:
            if timeout_count == max_retry:
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred for {max_retry} times, skipping row {start_row+row.name+1}. {e}")
                return None
            else:
                timeout_count += 1
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred, retrying after {timeout} seconds, skipping row {start_row+row.name+1} after {max_retry-timeout_count+1} more retries. {e}")
                time.sleep(timeout)

# Apply the function to each row to calculate the trip distance and save it in a new column
df_uberdata['trip_distance'] = df_uberdata.apply(calculate_trip_distance, axis=1)

# Export the dataframe to a CSV file with the current time in the file name
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
output_file = f"uber_dataset_driving_distance_rows_{start_row}-{end_row}_{current_time}.csv"
df_uberdata.to_csv(output_file, index=False)

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Dataframe exported to {output_file}")
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Finished processing rows {start_row}-{end_row}")
    
# Move on to the next chunk
start_row = end_row

## 6th Iteration

In [ ]:
end_row = start_row+rows_per_chunk # Value of end_row to enable resuming from last cell

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Loading CSV into DataFrame...")
df_uberdata = pd.read_csv(uberdata)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - CSV is loaded")
df_uberdata.insert(0, 'no', df_uberdata.reset_index().index + 1)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Added row index column")
df_uberdata = df_uberdata.iloc[start_row:end_row].reset_index(drop=True)
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Starting processing rows {start_row} - {end_row}")

# Define a function to calculate the driving distance between two coordinates
def calculate_trip_distance(row):
    current_time = datetime.datetime.now(timezone).strftime(datetime_format)
    print(f"{current_time} - Calculating distance for row {start_row+row.name+1}")
    pickup_coordinates = f"{row['pickup_latitude']}, {row['pickup_longitude']}"
    dropoff_coordinates = f"{row['dropoff_latitude']}, {row['dropoff_longitude']}"
    region = 'US'
    timeout_count = 0
    while True:
        try:
            route = WazeRouteCalculator.WazeRouteCalculator(pickup_coordinates, dropoff_coordinates, region)
            distance_km, travel_time = route.calc_route_info()  # Unpack the tuple
            return distance_km
        except Exception as e:
            if timeout_count == max_retry:
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred for {max_retry} times, skipping row {start_row+row.name+1}. {e}")
                return None
            else:
                timeout_count += 1
                current_time = datetime.datetime.now(timezone).strftime(datetime_format)
                print(f"{current_time} - Request timeout occurred, retrying after {timeout} seconds, skipping row {start_row+row.name+1} after {max_retry-timeout_count+1} more retries. {e}")
                time.sleep(timeout)

# Apply the function to each row to calculate the trip distance and save it in a new column
df_uberdata['trip_distance'] = df_uberdata.apply(calculate_trip_distance, axis=1)

# Export the dataframe to a CSV file with the current time in the file name
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
output_file = f"uber_dataset_driving_distance_rows_{start_row}-{end_row}_{current_time}.csv"
df_uberdata.to_csv(output_file, index=False)

current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Dataframe exported to {output_file}")
current_time = datetime.datetime.now(timezone).strftime(datetime_format)
print(f"{current_time} - Finished processing rows {start_row}-{end_row}")
    
# Move on to the next chunk
start_row = end_row